# Load dataset 

In [1]:
from pathlib import Path
!pip install numpy
import numpy as np
!pip install pandas
import pandas as pd
!pip install plotly
import plotly.express as px

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [20]:
df=pd.read_csv("../../annotations_metadata.csv")
df= df[df['label'] != "idk/skip"] # mostly unrelated garbage (chinese text, non-sentences)
df= df[df['label'] != "relation"] 
df["hate"] = df["label"].map({'noHate': False, 'hate': True})
df=df.drop(columns=['label'])
df

,file_id,user_id,subforum_id,num_contexts,hate
0,12834217_1,572066,1346,0,False
1,12834217_2,572066,1346,0,False
2,12834217_3,572066,1346,0,False
3,12834217_4,572066,1346,0,True
4,12834217_5,572066,1346,0,False
...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False
10940,33677019_1,735154,1388,0,False
10941,33677019_2,735154,1388,0,False
10942,33677053_1,572266,1388,0,True


In [21]:
from pathlib import Path
BASE_PATH = Path().parent.absolute()
def retrieve_text(file_id:str):
    filename=file_id+".txt"
    with open(Path(BASE_PATH)/ '..' / '..' / "all_files" / filename, encoding='utf-8') as f:
        return f.read()
def is_train(file_id:str):
    filename=file_id+".txt"
    return (Path(BASE_PATH) / '..' / '..' /"sampled_train"/filename).exists()
def is_test(file_id:str):
    filename=file_id+".txt"
    return (Path(BASE_PATH) /'..' / '..' /"sampled_test"/filename).exists()

df["text"] = df["file_id"].apply(retrieve_text)
df["train"] = df["file_id"].apply(is_train)
df["test"] = df["file_id"].apply(is_test)

In [22]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [53]:
train_df = df[df['train'] == True]
train_df = train_df[['text', 'hate']]
train_df['hate'] = train_df['hate'].astype(float)
train_df['text'] = 'human: ' + train_df['text']
train_df['text'] = train_df.apply(lambda row: row['text'] + f"  \n bot: ```json\n{{\n  \"hate_speech_probability\": {row['hate']:.2f},\n}}\n```", axis=1)
train_df = train_df[['text']]
train_df

,text
1,human: In order to help increase the booklets ...
3,human: Click below for a FREE download of a co...
10,human: She may or may not be a Jew but she 's ...
20,human: I think you should write a book as well...
23,human: I guess I 'll just stick to homeschooli...
...,...
10923,human: Check it out this is disturbing but ins...
10930,human: I can only pray this babboon mongoloid ...
10932,human: Reports that Trump will meet with group...
10933,human: Trudeau is the white Obama as he does n...


In [54]:
print(train_df.iloc[1, 0])

human: Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization .  
 bot: ```json
{
  "hate_speech_probability": 1.00,
}
```


In [55]:
df.to_pickle('processed_data.pkl')
train_df.to_csv('train_data.csv', index=False)